In [15]:
import pandas as pd

df=pd.read_csv('h.csv',sep=';',header=None).dropna()

In [16]:
df['StockCode']=df[0]
df['Description']=df[1]
df['Brand']=df[2]
df['Category']=df[3]
df['SM_Comm']=df[4]
df['Specialist_Comm']=df[5]
df['SM_Comm_Promo']=df[6]
df['Specialist_Comm_Promo']=df[7]

In [17]:
df1=df[['StockCode', 'Description', 'Brand', 'Category', 'SM_Comm',
       'Specialist_Comm', 'SM_Comm_Promo', 'Specialist_Comm_Promo']].fillna('NULL')

In [18]:
df1

,StockCode,Description,Brand,Category,SM_Comm,Specialist_Comm,SM_Comm_Promo,Specialist_Comm_Promo
0,KMW112,BR 20L DIGITAL MICROWAVE OVEN,BR MICROWAVES,LARGE APPLIANCES,0.25%,0.25%,0.25%,0.25%
1,KMW101,BR 28L MICROWAVE,BR MICROWAVES,LARGE APPLIANCES,0.50%,1.00%,1.00%,1.00%
2,KMW102,BR 28L DIGITAL MICROWAVE OVEN,BR MICROWAVES,LARGE APPLIANCES,0.50%,1.00%,1.00%,1.00%


In [19]:
import pyodbc
import time
time.clock = time.time
cnxn3 = pyodbc.connect('Driver={SQL Server};'
                      'Server=tevdbn-sql-01;'
                      'Database=SysproCompany1;'
                      'Trusted_Connection=yes;'
)

# SQL query
sql_query3 = 'select StockCode,Brand,Name,Category,SM_Comm,Specialist_Comm,SM_Comm_Promo,Specialist_Comm_Promo from [InvMaster+]'
# Execute the query and fetch results into a DataFrame
df4 = pd.read_sql(sql_query3, cnxn3)

# Close the connection
cnxn3.close()

C:\Users\shraDDhar\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [20]:
t=[i for i in df1['StockCode'].values.tolist() if i in df4['StockCode'].values.tolist()]
len(t)==len(df1)

True

In [21]:
df4['StockCode']=[i.upper() for i in df4['StockCode'].values.tolist() ]

In [22]:
df1['SM_Comm'] = df1['SM_Comm'].str.replace('%',"").astype(float)/100
df1['Specialist_Comm'] = df1['Specialist_Comm'].str.replace('%',"").astype(float)/100
df1['SM_Comm_Promo'] = df1['SM_Comm_Promo'].str.replace('%',"").astype(float)/100
df1['Specialist_Comm_Promo'] = df1['Specialist_Comm_Promo'].str.replace('%',"").astype(float)/100

In [23]:
cnxn3 = pyodbc.connect('Driver={SQL Server};'
                      'Server=tevdbn-sql-01;'
                      'Database=SysproCompany1;'
                      'Trusted_Connection=yes;'
)
for i in df1['StockCode'].values.tolist():
    if i in df4['StockCode'].values.tolist():
        p1=df1['StockCode'].values.tolist().index(i)
        update_query = "UPDATE [InvMaster+] SET SM_Comm =" +str(df1['SM_Comm'].values.tolist()[p1])+ ",Specialist_Comm ="+ str(df1['Specialist_Comm'].values.tolist()[p1])+",SM_Comm_Promo = "+ str(df1['SM_Comm_Promo'].values.tolist()[p1])+",Specialist_Comm_Promo = "+ str(df1['Specialist_Comm_Promo'].values.tolist()[p1])+"""FROM [InvMaster+] WHERE [InvMaster+].StockCode =\'"""+i+"\'"
        cnxn3.execute(update_query)
        cnxn3.commit()
cnxn3.close()

In [11]:
df1.join(df4,how='left',on='StockCode')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [12]:
sv=[]
for i in df1['StockCode'].values.tolist():
    sv.append(" StockCode=\'"+i+"\' and ")

In [13]:
" ".join(sv)

" StockCode='KMW112' and   StockCode='KMW101' and   StockCode='KMW102' and "

In [14]:
update_query

"UPDATE [InvMaster+] SET SM_Comm =0.005,Specialist_Comm =0.01,SM_Comm_Promo = 0.01,Specialist_Comm_Promo = 0.01FROM [InvMaster+] WHERE [InvMaster+].StockCode ='KMW102'"